In [2]:
import numpy as np
import pandas as pd
import gc
import sys

aisles = pd.read_csv('./data/aisles.csv')
departments = pd.read_csv('./data/departments.csv')
order_products_prior = pd.read_csv('./data/order_products__prior.csv')
order_products_train = pd.read_csv('./data/order_products__train.csv')
orders = pd.read_csv('./data/orders.csv')
products = pd.read_csv('./data/products.csv')

shopping_cart = pd.concat(
    [
        order_products_prior,
        order_products_train
    ]
)


/home/li/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [3]:
# all the order_id in the prior and train
t = pd.merge(
    orders,
    shopping_cart,
    on = 'order_id',
    how = 'inner'
)

del shopping_cart
del order_products_prior
del order_products_train
t.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Unnamed: 0,Unnamed: 0.1,add_to_cart_order,product_id,reordered
0,2539329,1,prior,1,2,8,NaN,608995.0,24076664.0,1,196,0
1,2539329,1,prior,1,2,8,NaN,10932048.0,24076667.0,4,26088,0
2,473747,1,prior,3,3,12,21.0,11068895.0,4488097.0,3,10258,1
3,2295261,1,prior,9,1,16,0.0,9383077.0,21760448.0,6,12427,1
4,1187899,1,train,11,4,8,14.0,NaN,NaN,1,196,1


In [4]:
order_id_and_number = t[['user_id', 'order_id', 'order_number','days_since_prior_order','order_dow', 'order_hour_of_day']].drop_duplicates()
order_id_and_number['days_since_prior_order'][order_id_and_number.days_since_prior_order.isnull()] = 0
order_id_and_number = order_id_and_number.sort_values(by = ['user_id', 'order_number'])
order_id_and_number.head()

,user_id,order_id,order_number,days_since_prior_order,order_dow,order_hour_of_day
0,1,2539329,1,0.0,2,8
2,1,473747,3,21.0,3,12
3,1,2295261,9,0.0,1,16
4,1,1187899,11,14.0,4,8
15,2,2168274,1,0.0,2,11


In [5]:
unique_users = order_id_and_number['user_id'].drop_duplicates().reset_index()
unique_users = unique_users.drop('index', axis = 1)

unique_users_sample = unique_users.sample(n = 1000)

In [8]:
order_id_and_number = pd.merge(
    order_id_and_number,
    unique_users_sample,
    on = 'user_id',
    how = 'inner'
)
order_id_and_number.head()

,user_id,order_id,order_number,days_since_prior_order,order_dow,order_hour_of_day
0,36,3285797,1,0.0,4,17
1,36,3067639,14,22.0,0,12
2,36,2640464,15,12.0,5,17
3,36,409708,17,11.0,0,15
4,36,423623,18,18.0,4,17


In [10]:
o = pd.merge(
    order_id_and_number,
    t,
    on = [
        'order_id',
        'user_id',
        'order_number',
        'order_dow',
        'order_hour_of_day'
    ],
    how = 'inner'
)

o = o.drop('days_since_prior_order_y', axis = 1)
o = o.rename({'days_since_prior_order_x':'days_since_prior_order'}, axis = 'columns')

o.head()

,user_id,order_id,order_number,days_since_prior_order,order_dow,order_hour_of_day,eval_set,Unnamed: 0,Unnamed: 0.1,add_to_cart_order,product_id,reordered
0,36,3285797,1,0.0,4,17,prior,15077412.0,31152342.0,10,32426,0
1,36,3067639,14,22.0,0,12,prior,12186778.0,29086701.0,2,44632,1
2,36,2640464,15,12.0,5,17,prior,8217958.0,25035419.0,8,26313,0
3,36,409708,17,11.0,0,15,prior,12238122.0,3882146.0,2,46995,1
4,36,409708,17,11.0,0,15,prior,6861344.0,3882148.0,4,39361,1


In [20]:
# COUNTS - HOUR / DAY
product_rank = t[['product_id', 'order_dow', 'order_hour_of_day']] \
    .groupby(['product_id', 'order_dow', 'order_hour_of_day']) \
    .size() \
    .reset_index(name = 'count')

product_rank.head()


,product_id,order_dow,order_hour_of_day,count
0,1,0,8,2
1,1,0,9,2
2,1,0,10,5
3,1,0,12,2
4,1,0,14,3


In [25]:
product_rank['count'][product_rank['count'].isnull()] = 0

product_rank['sum_help'] = 1
product_rank.head()

,product_id,order_dow,order_hour_of_day,count,sum_help
0,1,0,8,2,1
1,1,0,9,2,1
2,1,0,10,5,1
3,1,0,12,2,1
4,1,0,14,3,1


In [30]:
# RANKS - HOUR / DAY - MACRO LEVEL
product_rank = product_rank.sort_values(by = ['order_dow', 'order_hour_of_day','count'], ascending = [True, True, False])

product_rank['rank_macro_day_hour'] = product_rank \
    .groupby(['order_dow', 'order_hour_of_day']) \
    .cumsum()['sum_help']


# RANKS - HOUR / DAY - PRODUCT LEVEL
product_rank = product_rank.sort_values(by = ['product_id','count'], ascending = [True, False])
product_rank['rank_product_day_hour'] = product_rank \
    .groupby(['product_id']) \
    .cumsum()['sum_help']

product_rank = product_rank.drop('sum_help', axis = 1)
product_rank = product_rank.rename({'count' : 'count_day_hour'}, axis = 'columns')
product_rank.head()

,product_id,order_dow,order_hour_of_day,count_day_hour,rank_macro_day_hour,rank_product_day_hour
17,1,1,10,10,1257,1
76,1,5,11,10,831,2
60,1,4,11,9,778,3
16,1,1,9,8,1410,4
20,1,1,13,8,1327,5


In [33]:
products0 = products['product_id'].to_frame()
products0['key'] = 1
# COUNTS - DAY LEVEL
time0 = t[['order_dow']].drop_duplicates().reset_index().drop('index', axis = 1)
time0['key'] = 1


In [34]:

products2 = pd.merge(
    products0,
    time0,
    on = 'key',
    how = 'inner'
).drop('key', axis = 1)

product_rank_day = t[['product_id', 'order_dow']] \
    .groupby(['product_id', 'order_dow']) \
    .size() \
    .reset_index(name = 'count')

product_rank_day = pd.merge(
    products2,
    product_rank_day,
    on = ['product_id', 'order_dow'],
    how = 'left'
)

product_rank_day['count'][product_rank_day['count'].isnull()] = 0
product_rank_day['sum_help'] = 1

# RANKS - DAY LEVEL - MACRO LEVEL
product_rank_day = product_rank_day.sort_values(by = ['order_dow','count'], ascending = [True, False])

product_rank_day['rank_macro_day'] = product_rank_day \
    .groupby(['order_dow']) \
    .cumsum()['sum_help']

# RANKS - DAY LEVEL - PRODUCT LEVEL
product_rank_day = product_rank_day.sort_values(by = ['product_id','count'], ascending = [True, False])

product_rank_day['rank_product_day'] = product_rank_day \
    .groupby(['product_id']) \
    .cumsum()['sum_help']

product_rank_day = product_rank_day.drop('sum_help', axis = 1)
product_rank_day = product_rank_day.rename({'count' : 'count_day'}, axis = 'columns')

/home/li/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:


# COUNTS - HOUR LEVEL
time1 = t[['order_hour_of_day']].drop_duplicates().reset_index().drop('index', axis = 1)
time1['key'] = 1

products3 = pd.merge(
    products0,
    time1,
    on = 'key',
    how = 'inner'
).drop('key', axis = 1)

product_rank_hour = t[['product_id', 'order_hour_of_day']] \
    .groupby(['product_id', 'order_hour_of_day']) \
    .size() \
    .reset_index(name = 'count')

product_rank_hour = pd.merge(
    products3,
    product_rank_hour,
    on = ['product_id', 'order_hour_of_day'],
    how = 'left'
)

product_rank_hour['count'][product_rank_hour['count'].isnull()] = 0
product_rank_hour['sum_help'] = 1

# RANKS - HOUR LEVEL - MACRO LEVEL
product_rank_hour = product_rank_hour.sort_values(by = ['order_hour_of_day','count'], ascending = [True, False])

product_rank_hour['rank_macro_hour'] = product_rank_hour \
    .groupby(['order_hour_of_day']) \
    .cumsum()['sum_help']

# RANKS - HOUR LEVEL - PRODUCT LEVEL
product_rank_hour = product_rank_hour.sort_values(by = ['product_id','count'], ascending = [True, False])

product_rank_hour['rank_product_hour'] = product_rank_hour \
    .groupby(['product_id']) \
    .cumsum()['sum_help']

product_rank_hour = product_rank_hour.drop('sum_help', axis = 1)
product_rank_hour = product_rank_hour.rename({'count' : 'count_hour'}, axis = 'columns')
product_rank_hour.head()

/home/li/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,product_id,order_hour_of_day,count_hour,rank_macro_hour,rank_product_hour
3,1,11,39.0,1713,1
4,1,10,34.0,2020,2
10,1,13,25.0,2639,3
1,1,12,22.0,2927,4
14,1,14,22.0,3028,5


In [37]:
# COUNTS - HOUR LEVEL
product_rank_overall = t[['product_id']] \
    .groupby(['product_id']) \
    .size() \
    .reset_index(name = 'count')

product_rank_overall = pd.merge(
    products0,
    product_rank_overall,
    on = ['product_id'],
    how = 'left'
)

product_rank_overall['count'][product_rank_overall['count'].isnull()] = 0
product_rank_overall['sum_help'] = 1

# RANKS - OVERALL
product_rank_overall = product_rank_overall.sort_values(by = ['count'], ascending = [False])

product_rank_overall['rank_macro_overall'] = product_rank_overall \
    .cumsum()['sum_help']

product_rank_overall = product_rank_overall.drop(['sum_help', 'key'], axis = 1)
product_rank_overall = product_rank_overall.rename({'count' : 'count_overall'}, axis = 'columns')
product_rank_overall.head()

/home/li/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,product_id,count_overall,rank_macro_overall
24851,24852,66180.0,1.0
13175,13176,53393.0,2.0
21136,21137,37384.0,3.0
21902,21903,34017.0,4.0
47208,47209,28720.0,5.0


# cross_join = TRAIN dataset

In [39]:
first_order = o[['user_id', 'order_number', 'product_id']] \
    .groupby(['user_id', 'product_id']) \
    .min()['order_number'] \
    .reset_index(name = 'first_order')
first_order.head()

,user_id,product_id,first_order
0,36,1090,27
1,36,12145,29
2,36,15655,31
3,36,16759,18
4,36,23054,29


In [41]:

# next: user_products
user_products = o[['user_id', 'product_id']].drop_duplicates().reset_index().drop('index', axis = 1)
user_products.head()

,user_id,product_id
0,36,32426
1,36,44632
2,36,26313
3,36,46995
4,36,39361


In [40]:
# next: user_orders
user_orders = o[['user_id', 'order_number', 'order_dow', 'order_hour_of_day']].drop_duplicates().reset_index().drop('index', axis = 1)

user_orders.head()

,user_id,order_number,order_dow,order_hour_of_day
0,36,1,4,17
1,36,14,0,12
2,36,15,5,17
3,36,17,0,15
4,36,18,4,17


In [42]:
# then join on user_id, filter out when order_number < first_order.order_number
cross_join = pd.merge(
    pd.merge(
        user_products,
        user_orders,
        on = 'user_id',
        how = 'inner'
    ),
    first_order,
    on = ['user_id', 'product_id'],
    how = 'inner'
)

In [43]:
order_days = o[['user_id', 'order_number', 'days_since_prior_order']].drop_duplicates().reset_index().drop('index', axis = 1)

order_days['days_agg'] = order_days \
    .groupby('user_id') \
    .cumsum()['days_since_prior_order']
order_days.head()

,user_id,order_number,days_since_prior_order,days_agg
0,36,1,0.0,0.0
1,36,14,22.0,22.0
2,36,15,12.0,34.0
3,36,17,11.0,45.0
4,36,18,18.0,63.0


In [44]:
cross_join = pd.merge(
    cross_join,
    order_days,
    on = ['user_id', 'order_number'],
    how = 'inner'
)
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg
0,36,32426,1,4,17,1,0.0,0.0
1,36,44632,1,4,17,14,0.0,0.0
2,36,26313,1,4,17,15,0.0,0.0
3,36,46995,1,4,17,17,0.0,0.0
4,36,39361,1,4,17,17,0.0,0.0


In [45]:
product_orders = o[['user_id', 'order_number', 'product_id']]
product_orders['ordered'] = 1
product_orders.head()

/home/li/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,order_number,product_id,ordered
0,36,1,32426,1
1,36,14,44632,1
2,36,15,26313,1
3,36,17,46995,1
4,36,17,39361,1


In [46]:
cross_join = pd.merge(
    cross_join,
    product_orders,
    on = ['user_id', 'order_number', 'product_id'],
    how = 'left'
)
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered
0,36,32426,1,4,17,1,0.0,0.0,1.0
1,36,44632,1,4,17,14,0.0,0.0,NaN
2,36,26313,1,4,17,15,0.0,0.0,NaN
3,36,46995,1,4,17,17,0.0,0.0,NaN
4,36,39361,1,4,17,17,0.0,0.0,NaN


In [53]:
# contain all the first order_id
cross_join.loc[cross_join.ordered.isnull(), 'ordered'] = 0

cross_join = cross_join.sort_values(by = ['user_id', 'order_number'], ascending = True)
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered
0,36,32426,1,4,17,1,0.0,0.0,1.0
1,36,44632,1,4,17,14,0.0,0.0,0.0
2,36,26313,1,4,17,15,0.0,0.0,0.0
3,36,46995,1,4,17,17,0.0,0.0,0.0
4,36,39361,1,4,17,17,0.0,0.0,0.0


In [54]:
cross_join['count_previous_orders'] = cross_join \
    .groupby(['user_id', 'product_id']) \
    .cumsum()['ordered'] - cross_join['ordered']
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders
0,36,32426,1,4,17,1,0.0,0.0,1.0,0.0
1,36,44632,1,4,17,14,0.0,0.0,0.0,0.0
2,36,26313,1,4,17,15,0.0,0.0,0.0,0.0
3,36,46995,1,4,17,17,0.0,0.0,0.0,0.0
4,36,39361,1,4,17,17,0.0,0.0,0.0,0.0


In [56]:
cross_join['shift_indicator'] = cross_join \
    .groupby(['user_id', 'product_id'])['ordered'] \
    .shift(1) \
    .fillna(0)

cross_join.tail()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,shift_indicator
281087,206044,26670,11,0,1,8,24.0,57.0,0.0,2.0,1.0
281088,206044,4527,11,0,1,8,24.0,57.0,0.0,1.0,0.0
281089,206044,521,11,0,1,10,24.0,57.0,0.0,1.0,1.0
281090,206044,44632,11,0,1,11,24.0,57.0,1.0,0.0,0.0
281091,206044,46041,11,0,1,11,24.0,57.0,1.0,0.0,0.0


In [59]:
cross_join['last_order_number'] = 0

cross_join['prev_order_number'] = cross_join \
    .groupby(['user_id', 'product_id'])['order_number'] \
    .shift(1)
cross_join.tail()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,shift_indicator,last_order_number,prev_order_number
281087,206044,26670,11,0,1,8,24.0,57.0,0.0,2.0,1.0,0,10.0
281088,206044,4527,11,0,1,8,24.0,57.0,0.0,1.0,0.0,0,10.0
281089,206044,521,11,0,1,10,24.0,57.0,0.0,1.0,1.0,0,10.0
281090,206044,44632,11,0,1,11,24.0,57.0,1.0,0.0,0.0,0,10.0
281091,206044,46041,11,0,1,11,24.0,57.0,1.0,0.0,0.0,0,10.0


In [60]:
cross_join.loc[cross_join.shift_indicator == 1, 'last_order_number'] = cross_join['prev_order_number']

In [61]:
cross_join['last_order_switch'] = 0
cross_join.loc[cross_join.last_order_number != 0, 'last_order_switch'] = 1
cross_join['bloc_party'] = cross_join \
    .groupby(['user_id', 'product_id']) \
    .cumsum()['last_order_switch']
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,shift_indicator,last_order_number,prev_order_number,last_order_switch,bloc_party
0,36,32426,1,4,17,1,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0,0
1,36,44632,1,4,17,14,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0
2,36,26313,1,4,17,15,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0
3,36,46995,1,4,17,17,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0
4,36,39361,1,4,17,17,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0


In [62]:
cross_join = pd.merge(
    cross_join,
    cross_join[['user_id', 'product_id', 'bloc_party', 'last_order_number']][cross_join.last_order_number != 0] \
        .drop_duplicates() \
        .reset_index(drop = True) \
        .rename({'last_order_number' : 'bloc_last_order_number'}, axis = 'columns'),
    on = ['user_id', 'product_id', 'bloc_party'],
    how = 'inner'
) \
    .drop('last_order_number', axis = 1) \
    .rename({'bloc_last_order_number' : 'last_order_number'}, axis = 'columns')

cross_join.head() 

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,shift_indicator,prev_order_number,last_order_switch,bloc_party,last_order_number
0,36,32426,14,0,12,1,22.0,22.0,0.0,1.0,1.0,1.0,1,1,1.0
1,36,32426,15,5,17,1,12.0,34.0,0.0,1.0,0.0,14.0,0,1,1.0
2,36,32426,17,0,15,1,11.0,45.0,0.0,1.0,0.0,15.0,0,1,1.0
3,36,32426,18,4,17,1,18.0,63.0,0.0,1.0,0.0,17.0,0,1,1.0
4,36,32426,24,2,7,1,5.0,68.0,0.0,1.0,0.0,18.0,0,1,1.0


In [65]:
cross_join = pd.merge(
    cross_join,
    order_days[['user_id', 'order_number', 'days_agg']].rename({'order_number' : 'last_order_number', 'days_agg' : 'days_agg_last'}, axis = 1),
    on = ['user_id', 'last_order_number'],
    how = 'inner'
)
cross_join['days_since_last_order'] = cross_join['days_agg'] - cross_join['days_agg_last']
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,shift_indicator,prev_order_number,last_order_switch,bloc_party,last_order_number,days_agg_last_x,days_agg_last_y,days_agg_last,days_since_last_order
0,36,32426,14,0,12,1,22.0,22.0,0.0,1.0,1.0,1.0,1,1,1.0,0.0,0.0,0.0,22.0
1,36,32426,15,5,17,1,12.0,34.0,0.0,1.0,0.0,14.0,0,1,1.0,0.0,0.0,0.0,34.0
2,36,32426,17,0,15,1,11.0,45.0,0.0,1.0,0.0,15.0,0,1,1.0,0.0,0.0,0.0,45.0
3,36,32426,18,4,17,1,18.0,63.0,0.0,1.0,0.0,17.0,0,1,1.0,0.0,0.0,0.0,63.0
4,36,32426,24,2,7,1,5.0,68.0,0.0,1.0,0.0,18.0,0,1,1.0,0.0,0.0,0.0,68.0


In [67]:
user_add_to_cart = o[['user_id', 'product_id', 'order_number', 'add_to_cart_order']]

total_orders = user_add_to_cart \
    .groupby(['user_id', 'order_number']) \
    .size() \
    .reset_index(name = 'num_products_in_order')

user_add_to_cart = pd.merge(
    user_add_to_cart,
    total_orders,
    on = ['user_id', 'order_number'],
    how = 'inner'
)

user_add_to_cart['pct_add_to_cart'] = user_add_to_cart['add_to_cart_order'] / user_add_to_cart['num_products_in_order']

user_add_to_cart['total_cart_order'] = user_add_to_cart \
    .groupby(['user_id', 'product_id']) \
    .cumsum()['pct_add_to_cart']

user_add_to_cart['order_count'] = 1

user_add_to_cart['num_orders'] = user_add_to_cart \
    .groupby(['user_id', 'product_id']) \
    .cumsum()['order_count']

# denominator: replace order_number with number orders so far
user_add_to_cart['avg_add_to_cart_order'] = user_add_to_cart['total_cart_order'] / user_add_to_cart['num_orders']


In [68]:
cross_join = pd.merge(
    cross_join,
    user_add_to_cart.rename({'order_number' : 'last_order_number'}, axis = 1),
    on = ['user_id', 'product_id', 'last_order_number'],
    how = 'inner'
)

cross_join = cross_join.sort_values(by = ['user_id', 'order_number'])

cross_join['num_orders_three_ago'] = cross_join \
    .groupby(['user_id', 'product_id'])['num_orders'] \
    .shift(3) \
    .fillna(0)

cross_join['num_orders_last_three'] =  cross_join['num_orders'] - cross_join['num_orders_three_ago']
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,...,days_since_last_order,add_to_cart_order,num_products_in_order,pct_add_to_cart,total_cart_order,order_count,num_orders,avg_add_to_cart_order,num_orders_three_ago,num_orders_last_three
0,36,32426,14,0,12,1,22.0,22.0,0.0,1.0,...,22.0,10,1,10.0,10.0,1,1,10.0,0.0,1.0
1,36,32426,15,5,17,1,12.0,34.0,0.0,1.0,...,34.0,10,1,10.0,10.0,1,1,10.0,0.0,1.0
8,36,44632,15,5,17,14,12.0,34.0,0.0,1.0,...,12.0,2,1,2.0,2.0,1,1,2.0,0.0,1.0
2,36,32426,17,0,15,1,11.0,45.0,0.0,1.0,...,45.0,10,1,10.0,10.0,1,1,10.0,0.0,1.0
9,36,44632,17,0,15,14,11.0,45.0,0.0,1.0,...,23.0,2,1,2.0,2.0,1,1,2.0,0.0,1.0


In [69]:


product_rank['product_id'] = product_rank['product_id'].astype(int)
product_rank['order_dow'] = product_rank['order_dow'].astype(int)
product_rank['order_hour_of_day'] = product_rank['order_hour_of_day'].astype(int)

product_rank_day['product_id'] = product_rank_day['product_id'].astype(int)
product_rank_day['order_dow'] = product_rank_day['order_dow'].astype(int)

product_rank_hour['product_id'] = product_rank_hour['product_id'].astype(int)
product_rank_hour['order_hour_of_day'] = product_rank_hour['order_hour_of_day'].astype(int)

product_rank_overall['product_id'] = product_rank_overall['product_id'].astype(int)

cross_join = pd.merge(
    cross_join,
    products,
    on = 'product_id',
    how = 'inner'
).drop('product_name', axis = 1)

cross_join = pd.merge(
    cross_join,
    product_rank,
    on = ['product_id', 'order_dow', 'order_hour_of_day'],
    how = 'inner'
)

cross_join = pd.merge(
    cross_join,
    product_rank_day,
    on = ['product_id', 'order_dow'],
    how = 'inner'
)

cross_join = pd.merge(
    cross_join,
    product_rank_hour,
    on = ['product_id', 'order_hour_of_day'],
    how = 'inner'
)

cross_join = pd.merge(
    cross_join,
    product_rank_overall,
    on = ['product_id'],
    how = 'inner'
)

In [70]:
cross_join['order_number'] = cross_join['order_number'].astype(int)
cross_join['product_id'] = cross_join['product_id'].astype(int)
cross_join['user_id'] = cross_join['user_id'].astype(int)

del o
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,...,rank_macro_day_hour,rank_product_day_hour,count_day,rank_macro_day,rank_product_day,count_hour,rank_macro_hour,rank_product_hour,count_overall,rank_macro_overall
0,36,32426,18,4,17,1,18.0,63.0,0.0,1.0,...,8488,16,2.0,20431,4,1.0,22823,8,18.0,18345.0
1,36,44632,15,5,17,14,12.0,34.0,0.0,1.0,...,35,67,1388.0,23,4,665.0,30,9,10925.0,25.0
2,147271,44632,37,5,17,16,4.0,194.0,0.0,2.0,...,35,67,1388.0,23,4,665.0,30,9,10925.0,25.0
3,181413,44632,6,0,17,5,19.0,116.0,0.0,1.0,...,37,21,2112.0,38,2,665.0,30,9,10925.0,25.0
4,36,44632,18,4,17,14,18.0,63.0,0.0,1.0,...,56,79,1229.0,24,7,665.0,30,9,10925.0,25.0


In [71]:
from sklearn.preprocessing import PolynomialFeatures

poly = cross_join[
    [
        'num_orders_last_three',
        'count_previous_orders',
        'total_cart_order',
        'days_since_last_order'
    ]
]

p = PolynomialFeatures(degree = 2).fit(poly)

poly = pd.DataFrame(
    data = p.transform(poly),
    columns = p.get_feature_names(poly.columns)
)

poly['user_id'] = cross_join['user_id']
poly['order_number'] = cross_join['order_number']
poly['product_id'] = cross_join['product_id']

cross_join = pd.merge(
    cross_join,
    poly,
    on = [
        'user_id', 
        'order_number', 
        'product_id',
        'num_orders_last_three',
        'count_previous_orders',
        'total_cart_order',
        'days_since_last_order'
    ],
    how = 'inner'
).drop('1', axis = 1)

cross_join.shape
cross_join.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,first_order,days_since_prior_order,days_agg,ordered,count_previous_orders,...,num_orders_last_three^2,num_orders_last_three count_previous_orders,num_orders_last_three total_cart_order,num_orders_last_three days_since_last_order,count_previous_orders^2,count_previous_orders total_cart_order,count_previous_orders days_since_last_order,total_cart_order^2,total_cart_order days_since_last_order,days_since_last_order^2
0,36,32426,18,4,17,1,18.0,63.0,0.0,1.0,...,0.0,0.0,0.000000,0.0,1.0,10.000000,63.0,100.000000,630.000000,3969.0
1,36,44632,15,5,17,14,12.0,34.0,0.0,1.0,...,1.0,1.0,2.000000,12.0,1.0,2.000000,12.0,4.000000,24.000000,144.0
2,147271,44632,37,5,17,16,4.0,194.0,0.0,2.0,...,1.0,2.0,7.000000,11.0,4.0,14.000000,22.0,49.000000,77.000000,121.0
3,181413,44632,6,0,17,5,19.0,116.0,0.0,1.0,...,1.0,1.0,6.666667,19.0,1.0,6.666667,19.0,44.444444,126.666667,361.0
4,36,44632,18,4,17,14,18.0,63.0,0.0,1.0,...,1.0,1.0,2.000000,41.0,1.0,2.000000,41.0,4.000000,82.000000,1681.0


In [75]:

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import f1_score


In [72]:

x = cross_join.copy().drop(
    [
        'ordered',
        'shift_indicator',
        'last_order_number',
        'add_to_cart_order',
        'first_order',
        'num_products_in_order',
        'bloc_party',
        'last_order_switch',
        'prev_order_number',
        'num_orders',
        'order_count',
        'num_orders_three_ago'
    ],
    axis = 1
)
y = cross_join['ordered']

In [76]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 0)

model = RandomForestClassifier()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

predictions = pd.DataFrame(
    data = predictions,
    columns = ['prediction']
)

y_test = y_test.reset_index()
y_test = y_test.drop('index', axis = 1)
predictions['actual'] = y_test

auc = roc_auc_score(y_test, predictions['prediction'])
print('AUC: ', auc)

f1 = f1_score(predictions['actual'], predictions['prediction'], average = 'binary')
print('f1: ', f1)

corr = cross_join.corr()
corr = corr['ordered'].to_frame().reset_index()
corr.columns = ['column', 'corr']

importances = model.feature_importances_
cols = x.columns
importances = pd.DataFrame(
    data = {'column' : cols, 'importance': importances}
)

importances = pd.merge(
    importances,
    corr,
    on = 'column',
    how = 'inner'
).sort_values(by = 'corr', ascending = False)
print(importances)

/home/li/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


AUC:  0.5007517428351743
f1:  0.005344021376085504
                                         column  importance      corr
30  num_orders_last_three count_previous_orders    0.003214  0.074276
29                      num_orders_last_three^2    0.001099  0.068725
15                        num_orders_last_three    0.001653  0.067514
7                         count_previous_orders    0.004266  0.058225
32  num_orders_last_three days_since_last_order    0.013610  0.055873
5                        days_since_prior_order    0.029058  0.048128
33                      count_previous_orders^2    0.003742  0.047374
21                                    count_day    0.033891  0.042295
27                                count_overall    0.029702  0.041962
18                               count_day_hour    0.028472  0.040946
24                                   count_hour    0.035156  0.039941
31       num_orders_last_three total_cart_order    0.013869  0.038056
10                                days_

In [77]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 0)

lgb_model = lgb.LGBMClassifier(
    n_estimators = 1000, 
    objective = 'binary', 
    class_weight = 'balanced', 
    learning_rate = 0.05, 
    reg_alpha = 0.1, 
    reg_lambda = 0.1, 
    subsample = 0.8, 
    n_jobs = -1, 
    random_state = 50
)

lgb_model.fit(
    x_train,
    y_train,
    eval_metric = 'auc'
)

p = lgb_model.predict_proba(x_test)
p_binary = lgb_model.predict(x_test)

lightGBM = roc_auc_score(y_test, p[:,1])
print('LGBM proba:', lightGBM)

lightGBM_binary = roc_auc_score(y_test, p_binary)
print('LGBM binary:', lightGBM_binary)

f1_lgbm = f1_score(y_test, p_binary, average = 'binary')
print('f1 binary: ', f1_lgbm)

lgbm_importances = lgb_model.feature_importances_
cols = x.columns
lgbm_importances = pd.DataFrame(
    data = {'column' : cols, 'importance': lgbm_importances}
).sort_values(by = 'importance', ascending = False)

print(lgbm_importances)

LGBM proba: 0.6606705065162675
LGBM binary: 0.5640658559009627
f1 binary:  0.1406025824964132
                                         column  importance
0                                       user_id        1913
6                                      days_agg        1853
1                                    product_id        1756
2                                  order_number        1722
20                        rank_product_day_hour        1680
37       total_cart_order days_since_last_order        1518
8                               days_agg_last_x        1487
11                        days_since_last_order        1293
16                                     aisle_id        1288
5                        days_since_prior_order        1220
19                          rank_macro_day_hour        1123
35  count_previous_orders days_since_last_order        1044
4                             order_hour_of_day         941
24                                   count_hour         933
12    

In [78]:
p = pd.DataFrame(
    data = p
)

p['actual'] = y_test

p['.1'] = 0
p.loc[p[1] >= .1, '.1'] = 1
lightGBM = roc_auc_score(y_test, p['.1'])
f1_lgbm = f1_score(y_test, p['.1'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .1:', lightGBM)

p['.2'] = 0
p.loc[p[1] >= .2, '.2'] = 1
lightGBM = roc_auc_score(y_test, p['.2'])
f1_lgbm = f1_score(y_test, p['.2'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .2:', lightGBM)

p['.3'] = 0
p.loc[p[1] >= .3, '.3'] = 1
lightGBM = roc_auc_score(y_test, p['.3'])
f1_lgbm = f1_score(y_test, p['.3'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .3:', lightGBM)

p['.4'] = 0
p.loc[p[1] >= .4, '.4'] = 1
lightGBM = roc_auc_score(y_test, p['.4'])
f1_lgbm = f1_score(y_test, p['.4'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .4:', lightGBM)

p['.5'] = 0
p.loc[p[1] >= .5, '.5'] = 1
lightGBM = roc_auc_score(y_test, p['.5'])
f1_lgbm = f1_score(y_test, p['.5'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .5:', lightGBM)

p['.6'] = 0
p.loc[p[1] >= .6, '.6'] = 1
lightGBM = roc_auc_score(y_test, p['.6'])
f1_lgbm = f1_score(y_test, p['.6'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .6:', lightGBM)

p['.7'] = 0
p.loc[p[1] >= .7, '.7'] = 1
lightGBM = roc_auc_score(y_test, p['.7'])
f1_lgbm = f1_score(y_test, p['.7'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .7:', lightGBM)

p['.8'] = 0
p.loc[p[1] >= .8, '.8'] = 1
lightGBM = roc_auc_score(y_test, p['.8'])
f1_lgbm = f1_score(y_test, p['.8'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .8:', lightGBM)

p['.9'] = 0
p.loc[p[1] >= .9, '.9'] = 1
lightGBM = roc_auc_score(y_test, p['.9'])
f1_lgbm = f1_score(y_test, p['.9'], average = 'binary')
print('f1: ', f1_lgbm)
print('LGBM: .9:', lightGBM)

f1:  0.10090629551707395
LGBM: .1: 0.5934241944996907
f1:  0.11369968581421544
LGBM: .2: 0.6096023880719995
f1:  0.12725631768953072
LGBM: .3: 0.6066019538082993
f1:  0.1359223300970874
LGBM: .4: 0.5855056502625973
f1:  0.1406025824964132
LGBM: .5: 0.5640658559009627
f1:  0.11823802163833075
LGBM: .6: 0.5376851874039144
f1:  0.07370184254606366
LGBM: .7: 0.5185612419016966
f1:  0.020956123117223315
LGBM: .8: 0.5046092908889963
f1:  0.0054757015742642025
LGBM: .9: 0.5013011095374482


# SET UP TEST DF

## Initial driver DataFrame

## Discovered cumsum() -> VERY useful tool!

In [ ]:
test = orders[orders.eval_set == 'test']
test_unique = test['user_id'].drop_duplicates().reset_index().drop('index', axis = 1)
test_unique_orders = pd.merge(
    test_unique,
    orders,
    on = 'user_id',
    how = 'inner'
)
test_unique_orders['days_since_prior_order'][test_unique_orders.days_since_prior_order.isnull()] = 0

test_unique_orders['days'] = test_unique_orders \
    .groupby('user_id') \
    .cumsum()['days_since_prior_order']

test_driver = pd.merge(
    test_unique,
    t,
    on = 'user_id',
    how = 'inner'
)

## Count of times product ordered previously per user

In [ ]:
test_counts = test_driver[['user_id', 'product_id']] \
    .groupby(['user_id', 'product_id']) \
    .size() \
    .reset_index(name = 'count_previous_orders')

test_counts.head()

## Days since last ordered

In [ ]:
test_last_order = test_driver[['user_id', 'product_id', 'order_number']] \
    .groupby(['user_id', 'product_id']) \
    .max()['order_number'] \
    .reset_index()

test_last_order = pd.merge(
    test[['user_id','order_number']],
    test_last_order,
    on = 'user_id',
    how = 'inner'
).rename({'order_number_x' : 'order_number_current', 'order_number_y' : 'order_number_last'}, axis = 'columns')

test_last_order = pd.merge(
    test_last_order,
    test_unique_orders[['user_id', 'order_number', 'days']],
    left_on = [
        'user_id',
        'order_number_current'
    ],
    right_on = [
        'user_id',
        'order_number'
    ],
    how = 'inner'
) \
    .rename({'days' : 'days_agg'}, axis = 'columns') \
    .drop('order_number', axis = 1)

test_last_order = pd.merge(
    test_last_order,
    test_unique_orders[['user_id', 'order_number', 'days']],
    left_on = [
        'user_id',
        'order_number_last'
    ],
    right_on = [
        'user_id',
        'order_number'
    ],
    how = 'inner'
) \
    .rename({'days' : 'days_agg_last'}, axis = 'columns') \
    .drop('order_number', axis = 1)

test_last_order['days_since_last_order'] = test_last_order['days_agg'] - test_last_order['days_agg_last']

test_last_order.head()

# Add to cart order

In [ ]:
add_to_cart_counts = test_driver[['user_id', 'order_number']] \
    .groupby(['user_id', 'order_number']) \
    .size() \
    .reset_index(name = 'total_cart_order')

add_to_cart = pd.merge(
    test_driver[['user_id', 'order_number', 'product_id','add_to_cart_order']],
    add_to_cart_counts,
    on = ['user_id', 'order_number'],
    how = 'inner'
)

add_to_cart['pct_add_to_cart'] = add_to_cart['add_to_cart_order'] / add_to_cart['total_cart_order']
last_order = add_to_cart.copy()

add_to_cart = add_to_cart[['user_id', 'product_id', 'pct_add_to_cart']] \
    .groupby(['user_id', 'product_id']) \
    .sum()['pct_add_to_cart'] \
    .reset_index(name = 'total_cart_order')

add_to_cart = pd.merge(
    add_to_cart,
    test_counts,
    on = ['user_id', 'product_id'],
    how = 'inner'
)

add_to_cart['avg_add_to_cart_order'] = add_to_cart['total_cart_order'] / add_to_cart['count_previous_orders']

# add most recent order's pct_add_to_cart
last_order_driver = last_order[['user_id', 'product_id', 'order_number']] \
    .groupby(['user_id', 'product_id']) \
    .max()['order_number'] \
    .reset_index(name = 'order_number')

last_order = pd.merge(
    last_order,
    last_order_driver,
    on = ['user_id', 'product_id', 'order_number'],
    how = 'inner'
)

add_to_cart = pd.merge(
    add_to_cart,
    last_order[['user_id', 'product_id', 'pct_add_to_cart']],
    on = ['user_id', 'product_id'],
    how = 'inner'
).drop('count_previous_orders', axis = 1)

add_to_cart.head()

In [ ]:
last_user_order = test_driver[['user_id', 'order_number']] \
    .groupby(['user_id']) \
    .max()['order_number'] \
    .reset_index(name = 'order_number_end')

last_user_order['order_number_start'] = last_user_order['order_number_end'] - 2

last_user_order.loc[last_user_order.order_number_start < 0, 'order_number_start'] = 0

recent_three_orders = pd.merge(
    test_driver,
    last_user_order,
    on = 'user_id',
    how = 'inner'
)

recent_three_orders = recent_three_orders[
    (recent_three_orders.order_number >= recent_three_orders.order_number_start) & 
    (recent_three_orders.order_number <= recent_three_orders.order_number_end)
]

recent_three_orders = recent_three_orders[['user_id', 'product_id']] \
    .groupby(['user_id', 'product_id']) \
    .size() \
    .reset_index(name = 'num_orders_last_three')

recent_three_orders

# Put it all together
## Features:
1. Count of times ordered per product (test_counts)
2. Days since product ordered (test_last_order)
3. days since original order (test_unique_orders)
4. days_since_prior_order
5. order_number
6. product_id
7. user_id
8. order_dow
9. order_hour_of_day

In [ ]:
test = pd.merge(
    test,
    test_counts,
    on = ['user_id'],
    how = 'inner'
)

test = pd.merge(
    test,
    test_last_order,
    left_on = ['user_id', 'product_id', 'order_number'],
    right_on = ['user_id', 'product_id', 'order_number_current'],
    how = 'inner'
)

test = pd.merge(
    test,
    products,
    on = 'product_id',
    how = 'inner'
).drop('product_name', axis = 'columns')

test = pd.merge(
    test,
    add_to_cart,
    on = ['user_id', 'product_id'],
    how = 'inner'
)

test = pd.merge(
    test,
    product_rank,
    on = ['product_id', 'order_dow', 'order_hour_of_day'],
    how = 'inner'
)

test = pd.merge(
    test,
    product_rank_day,
    on = ['product_id', 'order_dow'],
    how = 'inner'
)

test = pd.merge(
    test,
    product_rank_hour,
    on = ['product_id', 'order_hour_of_day'],
    how = 'inner'
)

test = pd.merge(
    test,
    product_rank_overall,
    on = ['product_id'],
    how = 'inner'
)

test = pd.merge(
    test,
    recent_three_orders,
    on = ['user_id', 'product_id'],
    how = 'left'
)

test['num_orders_last_three'][test.num_orders_last_three.isnull()] = 0

test = test.drop(
    [
        'eval_set',
        'order_number_current',
        'order_number_last',
        'order_id'
    ],
    axis = 1
)

# Polynomial Features

In [ ]:
x.dtypes

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_test = test[
    [
        'num_orders_last_three',
        'count_previous_orders',
        'total_cart_order',
        'days_since_last_order'
    ]
]

p_test = PolynomialFeatures(degree = 2).fit(poly_test)

poly_test = pd.DataFrame(
    data = p_test.transform(poly_test),
    columns = p_test.get_feature_names(poly_test.columns)
)

poly_test['user_id'] = test['user_id']
poly_test['order_number'] = test['order_number']
poly_test['product_id'] = test['product_id']

test = pd.merge(
    test,
    poly_test,
    on = [
        'user_id', 
        'order_number', 
        'product_id',
        'num_orders_last_three',
        'count_previous_orders',
        'total_cart_order',
        'days_since_last_order'
    ],
    how = 'inner'
).drop('1', axis = 1)

test.shape

In [ ]:
test = test[
    [
        'user_id',
        'product_id',
        'order_number',
        'order_dow',
        'order_hour_of_day',
        'days_since_prior_order',
        'days_agg',
        'count_previous_orders',
        'days_agg_last',
        'days_since_last_order',
        'pct_add_to_cart',
        'total_cart_order',
        'avg_add_to_cart_order',
        'num_orders_last_three',
        'aisle_id',
        'department_id',
        'count_day_hour',
        'rank_macro_day_hour',
        'rank_product_day_hour',
        'count_day',
        'rank_macro_day',
        'rank_product_day',
        'count_hour',
        'rank_macro_hour',
        'rank_product_hour',
        'count_overall',
        'rank_macro_overall',
        'num_orders_last_three^2',
        'num_orders_last_three count_previous_orders',
        'num_orders_last_three total_cart_order',
        'num_orders_last_three days_since_last_order',
        'count_previous_orders^2',
        'count_previous_orders total_cart_order',
        'count_previous_orders days_since_last_order',
        'total_cart_order^2',
        'total_cart_order days_since_last_order',
        'days_since_last_order^2'
    ]
]

# Create submission file!

In [ ]:
test['prediction'] = lgb_model.predict_proba(test)[:,1]

submit_staging = test[['user_id', 'product_id', 'order_number']][test.prediction >= .7]

submit_staging = pd.merge(
    submit_staging,
    orders[['user_id', 'order_number', 'order_id']],
    on = ['user_id', 'order_number'],
    how = 'inner'
)

test_left_side = orders[orders.eval_set == 'test']
submit_staging2 = pd.merge(
    test_left_side[['user_id', 'order_id']],
    submit_staging,
    on = ['user_id', 'order_id'],
    how = 'left'
)

submit_staging3 = submit_staging2[['order_id', 'product_id']]
submit_staging3['product_id'][submit_staging3.product_id.isnull()] = 0
submit_staging3 = submit_staging3.astype(int)
submit_staging3 = submit_staging3.astype(str)

submit = submit_staging3 \
    .groupby('order_id')['product_id'] \
    .apply(' '.join) \
    .reset_index(name = 'products')

submit.loc[submit.products == '0', 'products'] = 'None'

submit.to_csv('submit.csv', index = False)